In [ ]:
# AIRE Micro Tutor Identity Cell
import datetime as dt
learner_id = input('Enter your learner ID (email or handle): ').strip()
learner_role = input('Enter your learner role (e.g., researcher, educator): ').strip()
resource_id = 'notebook:text_cleaning'
session_id = f"{learner_id}-{dt.datetime.utcnow().isoformat()}"

In [ ]:
# AIRE Micro Tutor Imports
from prompt_tutor import evaluate_prompt
from aire_telemetry import log_event
from resources_map import RESOURCE_MAP

In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# Ingest and Clean Synthetic Articles

**What**: Normalize and prepare synthetic article titles and abstracts for downstream analysis.

**Why**: Text data often contains noise that can degrade the performance of clustering and retrieval algorithms. Establishing a clean baseline is a critical first step in any NLP pipeline.

**How**:
1. **Install dependencies** (if running in Colab).
2. **Load data** from the synthetic dataset.
3. **Apply cleaning functions** to normalize text (lowercase, remove special characters).
4. **Verify** the output structure.

**Key Concept**: **Normalization** is the process of transforming text into a standard format (e.g., lowercase, no punctuation) to ensure consistency during analysis.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You loaded synthetic articles.
- You produced cleaned text and saw the shape of your dataset.
- You have a DataFrame ready for clustering or retrieval.

In [ ]:
from pathlib import Path


def find_data_dir() -> Path:
    candidates = [Path.cwd() / "data", Path.cwd().parent / "data", Path.cwd().parent.parent / "data"]
    for candidate in candidates:
        if (candidate / "sample_texts" / "articles_sample.csv").exists():
            return candidate
    raise FileNotFoundError("data directory not found. Run scripts/generate_synthetic_data.py.")

DATA_DIR = find_data_dir()


In [ ]:
import pandas as pd

articles_path = DATA_DIR / "sample_texts" / "articles_sample.csv"
articles = pd.read_csv(articles_path)
print(f"Loaded {len(articles)} articles from {articles_path}")
articles.head()


## Basic cleaning

In [ ]:
import re

def clean_text(text: str) -> str:
    text = text.lower().strip()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    return " ".join(text.split())

articles["cleaned"] = articles["abstract"].apply(clean_text)
articles[["title", "cleaned"]].head()


### If you get stuck / What to try next

If you get stuck: rerun the first Colab setup cell and ensure `scripts/generate_synthetic_data.py` has been run. What to try next: cluster cleaned abstracts in pipelines/text/clustering_and_topics.ipynb (see docs/colab_index.md for a Colab badge).

In [ ]:
# AIRE Micro Tutor Prompt + Feedback Cell
from aire_llm_client import chat_completion

def call_llm(prompt_text: str) -> str:
    messages = [
        {'role': 'system', 'content': 'You are an AI helper responding concisely.'},
        {'role': 'user', 'content': prompt_text},
    ]
    return chat_completion(messages)

prompt_text = input('Enter the prompt you want to test: ').strip()
ai_response = call_llm(prompt_text)
evaluation = evaluate_prompt(prompt_text, learner_role)

clarity = evaluation.get('clarity_score')
context = evaluation.get('context_score')
constraints = evaluation.get('constraints_score')
evaluation_score = evaluation.get('evaluation_score')
primary_weakness = evaluation.get('primary_weakness', '')
recommended_resource_id = RESOURCE_MAP.get(primary_weakness) or RESOURCE_MAP.get('evaluation')

log_event(
    event_name='micro_tutor_evaluation',
    user_id=learner_id or session_id,
    metadata={
        'resource_id': resource_id,
        'learner_role': learner_role,
        'clarity': clarity,
        'context': context,
        'constraints': constraints,
        'evaluation_score': evaluation_score,
        'primary_weakness': primary_weakness,
    },
)

print('\n--- AI Response ---')
print(ai_response)
print('\n--- Feedback ---')
print(evaluation.get('summary', 'No summary provided.'))
print('Primary weakness:', primary_weakness or 'n/a')
print('Strengths:', ', '.join(evaluation.get('strengths', [])) or 'n/a')
print('Suggestions:', ', '.join(evaluation.get('suggestions', [])) or 'n/a')
print('\nRecommended resource:', recommended_resource_id or 'n/a')